In [3]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint
import re
import time

In [4]:
house_dict = []

# Filter Out Coming Soon, Auction, Home Type: Apartment, Days On Zillow: 90 Days 
linked = 'https://www.zillow.com/ca/duplex/?searchQueryState={"pagination":{},"usersSearchTerm":"ca","mapBounds":{"west":-152.089851125,"east":-86.523444875,"south":14.485661126627711,"north":55.01789701556767},"regionSelection":[{"regionId":9,"regionType":2}],"isMapVisible":true,"mapZoom":4,"filterState":{"isPreMarketForeclosure":{"value":false},"isPreMarketPreForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isSingleFamily":{"value":false},"isManufactured":{"value":false},"isCondo":{"value":false},"isMultiFamily":{"value":false},"isLotLand":{"value":false},"isTownhouse":{"value":false},"doz":{"value":"90"}},"isListVisible":true}'
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKait/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'
headers = {'User-Agent': user_agent}
response = requests.get(linked, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
results = soup.find_all('article', class_='list-card')
    
# Loop through returned results
for result in results:
#     print(result)
    # Error handling
    try:
        # Identify and return housing weblink
        house_website = str(result.a.get('href'))
        print(house_website)
        # Identify and return house data
        listing_type = result.find('div', class_="list-card-footer").text;
        price = result.find('div', class_="list-card-price").text
#         print(price);
        full_address = result.find('address', class_="list-card-addr").text
        street = result.find('address', class_="list-card-addr").text.split(",")[0]
        city = result.find('address', class_="list-card-addr").text.split(",")[1].replace(" ", "")
        state = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[1]
        try:
            zip_code = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[2]
        except IndexError:
            zip_code = 'null'
        bed = result.find('ul', class_="list-card-details").find('li').text.split(" ")[0]
        bath = result.find('ul', class_="list-card-details").select('li')[1].text.split(" ")[0]
        house_square_feet = result.find('ul', class_="list-card-details").select('li')[2].text.split(" ")[0]
        # Loop inside each house website and get more house data
        loop_response = requests.get(house_website, headers=headers)
        loop_soup = BeautifulSoup(loop_response.text, 'html.parser')
        
        loop_result1 = loop_soup.find('ul', class_="ds-home-fact-list")
        loop_result2 = loop_soup.findAll('li', class_="ds-home-fact-list-item")
        
        lot = 'null';
        house_type = 'null';
        year_build = 'null';
        parking = 'null';
        hoa = 'null';
        lot_conversion_to_sqft = 'null';
        zillow_price_per_sqft = 'null';

        for content in loop_result2:
            if "Lot" in content.text.split(":")[0]:
                lot = content.text.split(":")[1];
            if "Type" in content.text.split(":")[0]:
                house_type = content.text.split(":")[1];
            if "Year built" in content.text.split(":")[0]:
                year_build = content.text.split(":")[1];
            if "Parking" in content.text.split(":")[0]:
                parking = content.text.split(":")[1];
            if "HOA" in content.text.split(":")[0]:
                hoa = content.text.split(":")[1];
            if "Price/sqft" in content.text.split(":")[0]:
                zillow_price_per_sqft = content.text.split(":")[1];
        try:
            if (str(lot.split(" ")[1]) == "sqft"):
                lot_conversion_to_sqft = float(lot.split(" ")[0].replace(',', ""));
            elif ((str(lot.split(" ")[1]) == "acres") or (str(lot.split(" ")[1]) == "acre")) or (str(lot.split(" ")[1]) == "Acres"):
                lot_conversion_to_sqft = round(float(lot.split(" ")[0].replace(',', "")) * 43560, 2);
            elif (lot == "No Data"):
                lot_conversion_to_sqft = 'null'
            else:
                lot_conversion_to_sqft = 'null'
        except IndexError:
            lot_conversion_to_sqft = 'null'
#         print(loop_soup.findAll('span', class_="Text-c11n-8-18-0__aiai24-0 einFCw"))
        try:
            zestimate = loop_soup.findAll('span', class_="Text-c11n-8-33-0__aiai24-0 LoSpP")[0].text
        except:
            zestimate = 'null'
        if (zestimate[0] != '$'):
            zestimate = 'null';
        print(zestimate, 'test')
#                 print('zestimate', zestimate)
        try:
            rent_zestimate = loop_soup.findAll('span', class_="Text-c11n-8-33-0__aiai24-0 LoSpP")[1].text
        except:
            rent_zestimate = 'null'
        print(rent_zestimate, 'test')

#                 print('zestimate', zestimate)
#             rent_zestimate = 'null'
#             print(zestimate)

            # Store data to house object if all elements are available
#             if (house_website and listing_type and price and full_address and street and city and state and zip_code and bed and bath and house_square_feet and house_type and year_build):
        house_obj = {
            "Listing_type": listing_type,
            "Price": price,
            "Full_address": full_address,
            "Street": street,
            "City": city,
            "State": state,
            "Zip_code": zip_code,
            "Bed": bed,
            "Bath": bath,
            "Square_Feet": house_square_feet,
            "Lot": lot,
            "Lot_conversion_to_sqft": lot_conversion_to_sqft,
            "House_type": "Apartment",
            "Year_build": year_build,
            "Parking": parking,
            "HOA": hoa,
            "Zillow_price/sqft": zillow_price_per_sqft,
            "Zestimate": zestimate,
            "Rent_zestimate": rent_zestimate,
            "House_website": house_website
            }

        #Continue to add data into collection house_dict
        house_dict.append(house_obj)
#         print(house_dict)
        time.sleep(3)
    except AttributeError as e:
        continue

print(house_dict)

https://www.zillow.com/homedetails/16934-Laurel-Hill-Ln-UNIT-165-San-Diego-CA-92127/67404429_zpid/
$2,700/mo test
null test
https://www.zillow.com/homedetails/1251-Parker-Pl-4-San-Diego-CA-92109/2088693667_zpid/
$573,323 test
$2,495/mo test
https://www.zillow.com/homedetails/690-Cassaday-Ct-San-Jose-CA-95136/19706480_zpid/
$2,143/mo test
null test
https://www.zillow.com/homedetails/606-Linden-Ct-Tehachapi-CA-93561/2099382236_zpid/
$900/mo test
null test
https://www.zillow.com/homedetails/606-Linden-Ct-10-Tehachapi-CA-93561/2071784640_zpid/
null test
null test
https://www.zillow.com/homedetails/849-Pine-Ave-Long-Beach-CA-90813/2132626049_zpid/
$458,038 test
$2,500/mo test
https://www.zillow.com/homedetails/1815-Old-Mammoth-Rd-Mammoth-Lakes-CA-93546/19228012_zpid/
$2,650/mo test
null test
https://www.zillow.com/homedetails/1651-W-215th-St-Torrance-CA-90501/21267603_zpid/
$801,948 test
$2,900/mo test
https://www.zillow.com/homedetails/425-Orange-St-APT-315-Oakland-CA-94610/24743453_zpid/


In [5]:
house_data = pd.DataFrame(house_dict)
house_data

,Listing_type,Price,Full_address,Street,City,State,Zip_code,Bed,Bath,Square_Feet,Lot,Lot_conversion_to_sqft,House_type,Year_build,Parking,HOA,Zillow_price/sqft,Zestimate,Rent_zestimate,House_website
0,,"$725,000","16934 Laurel Hill Ln UNIT 165, San Diego, CA 9...",16934 Laurel Hill Ln UNIT 165,SanDiego,CA,92127,2,3,"1,445",null,null,Apartment,2006,"Garage - Attached, Garage - Detached, Off-stre...",$185 monthly,$502,"$2,700/mo",null,https://www.zillow.com/homedetails/16934-Laure...
1,,"$500,000","1251 Parker Pl #4, San Diego, CA 92109",1251 Parker Pl #4,SanDiego,CA,92109,1,1,738,null,null,Apartment,1974,"Garage - Attached,",$400 monthly,$678,"$573,323","$2,495/mo",https://www.zillow.com/homedetails/1251-Parker...
2,,"$455,000","690 Cassaday Ct, San Jose, CA 95136",690 Cassaday Ct,SanJose,CA,95136,1,1,648,null,null,Apartment,1988,"Carport, Off-street,",$460 monthly,$702,"$2,143/mo",null,https://www.zillow.com/homedetails/690-Cassada...
3,,"$820,000","606 Linden Ct, Tehachapi, CA 93561",606 Linden Ct,Tehachapi,CA,93561,10,10,400,null,null,Apartment,1964,Garage - Detached,null,"$2,050",$900/mo,null,https://www.zillow.com/homedetails/606-Linden-...
4,,"$820,000","606 Linden Ct #10, Tehachapi, CA 93561",606 Linden Ct #10,Tehachapi,CA,93561,10,10,"4,480",null,null,Apartment,1964,0 spaces,null,$183,null,null,https://www.zillow.com/homedetails/606-Linden-...
5,,"$455,000","849 Pine Ave, Long Beach, CA 90813",849 Pine Ave,LongBeach,CA,90813,Studio,--,"1,450",null,null,Apartment,1926,No Data,null,$314,"$458,038","$2,500/mo",https://www.zillow.com/homedetails/849-Pine-Av...
6,,"$1,400,000","1815 Old Mammoth Rd, Mammoth Lakes, CA 93546",1815 Old Mammoth Rd,MammothLakes,CA,93546,9,5,"2,600",null,null,Apartment,1963,Off-street,null,$538,"$2,650/mo",null,https://www.zillow.com/homedetails/1815-Old-Ma...
7,,"$761,000","1651 W 215th St, Torrance, CA 90501",1651 W 215th St,Torrance,CA,90501,Studio,--,355,null,null,Apartment,1944,"Off-street, , Garage",$350 monthly,"$2,144","$801,948","$2,900/mo",https://www.zillow.com/homedetails/1651-W-215t...
8,,"$429,000","425 Orange St APT 315, Oakland, CA 94610",425 Orange St APT 315,Oakland,CA,94610,1,1,559,null,null,Apartment,1966,"Garage, Garage - Attached, Off-street, Covered",$521 monthly,$767,"$1,995/mo",null,https://www.zillow.com/homedetails/425-Orange-...
9,,"$526,000","1143 E Alvarado St, Pomona, CA 91767",1143 E Alvarado St,Pomona,CA,91767,Studio,--,"1,175",null,null,Apartment,1955,No Data,null,$448,"$530,267","$2,500/mo",https://www.zillow.com/homedetails/1143-E-Alva...


In [6]:
# Export file as a CSV, without the Pandas index, but with the header
house_data.to_csv("Output/Zillow/Q1-2021_Apartment_page1.csv", index=False, header=True)